In [5]:
from WindowsOpt import *

In [6]:
## configurations

# ## For a single-channel WSOlayer
# nch_window = 1
# act_window = "sigmoid"
# upbound_window = 255.0
# init_windows = "brain"

## For multi-channel WSOlayer
nch_window = 2
act_window = "sigmoid"
upbound_window = 255.0
init_windows = "ich_init"

optimizer = SGD(lr=0.0001, decay=0, momentum=0.9, nesterov=True)
input_shape = (512, 512, 1)
input_tensor = keras.layers.Input(shape=input_shape, name="input")


In [7]:
#### NOTE
## Define a window setting optimization layer
x = WindowOptimizer(nch_window=nch_window, act_window=act_window, upbound_window=upbound_window,
                    kernel_initializer="he_normal",
                    kernel_regularizer=regularizers.l2(0.5 * 1e-5)
                    )(input_tensor)

## ... add some your layer here
x = Conv2D(32, (3, 3), activation=None, padding="same", name="conv1")(x)
x = Activation("relu", name="conv1_relu")(x)
x = MaxPooling2D((7, 7), strides=(3, 3), name="pool1")(x)
x = Conv2D(256, (3, 3), activation=None, padding="same", name="conv2")(x)
x = Activation("relu", name="conv2_relu")(x)
x = MaxPooling2D((7, 7), strides=(3, 3), name="pool2")(x)
x = GlobalAveragePooling2D(name="gap")(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=outputs, name="main_model")


WindowOptimze kwargs :  {'kernel_initializer': 'he_normal', 'kernel_regularizer': <keras.regularizers.L1L2 object at 0x7f706465c6d8>}


In [8]:

#### NOTE
## Initialize parameters of window setting opt module
model = initialize_window_setting(model, act_window=act_window, init_windows=init_windows, upbound_window=upbound_window)


In [9]:

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

## Double check initialized parameters for WSO
names = [weight.name for layer in model.layers for weight in layer.weights]
# print(names)
weights = model.get_weights()

for name, weight in zip(names, weights):
    if "window_conv" in name:
        if "kernel:0" in name:
            ws = weight
        if "bias:0" in name:
            bs = weight

print("window optimization modeul set up (initialized with {} settings)".format(init_windows))
print("(WL, WW)={}".format(dict_window_settings[init_windows]))
print("Loaded parameter : w={} b={}".format(ws[0, 0, 0, :], bs)) # check result
print("Expected paramter(brain) : w=[0.11074668] b=[-5.5373344]")
print("Expected paramter(subdural) : w=[0.08518976] b=[-4.259488]")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 512, 512, 1)       0         
_________________________________________________________________
window_conv (Conv2D)         (None, 512, 512, 2)       4         
_________________________________________________________________
window_act (Activation)      (None, 512, 512, 2)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 512, 512, 32)      608       
_________________________________________________________________
conv1_relu (Activation)      (None, 512, 512, 32)      0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 169, 169, 32)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 169, 169, 256)     73984     
__________